In [32]:
import pandas as pd
import yfinance as yf
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Valid Time Periods: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max

data_ticker = yf.download("RIVN", period="5y")
# data_corr = yf.download("SCCO", period="5y")
# data_industry = yf.download("COPX", period="5y")
# data_P_index = yf.download("HG=F", period="5y")
# data_S_index = yf.download("GC=F", period="5y")

data_ticker["Percent Change"] = data_ticker["Close"].pct_change() * 100
data_ticker = data_ticker.dropna(subset=["Percent Change"])


data_ticker["5 Day Percent Change"] = ((data_ticker["Close"].shift(-5) - data_ticker["Close"])/data_ticker["Close"]) * 100
data_ticker = data_ticker.dropna(subset=["5 Day Percent Change"])

# data_P_index['Percent Change'] = data_P_index['Close'].pct_change() * 100
# data_P_index = data_P_index.dropna(subset=['Percent Change'])

# data_S_index['Percent Change'] = data_S_index['Close'].pct_change() * 100
# data_S_index = data_S_index.dropna(subset=['Percent Change'])


# data_corr['Percent Change'] = data_corr['Close'].pct_change() * 100
# data_corr = data_corr.dropna(subset=['Percent Change'])


# data_industry['Percent Change'] = data_industry['Close'].pct_change() * 100
# data_industry = data_industry.dropna(subset=['Percent Change'])

n_days = 10
candlestick_data = pd.DataFrame()

for i in range(1, n_days + 1):
    candlestick_data[f"Day_{i} Ticker Open"] = data_ticker["Open"].shift(i)
    candlestick_data[f"Day_{i} Ticker High"] = data_ticker["High"].shift(i)
    candlestick_data[f"Day_{i} Ticker Low"] = data_ticker["Low"].shift(i)
    candlestick_data[f"Day_{i} Ticker Adj Close"] = data_ticker["Adj Close"].shift(i)
    candlestick_data[f"Day_{i} Ticker Volume"] = data_ticker["Volume"].shift(i)
    candlestick_data[f"Day_{i} Ticker Percent Change"] = data_ticker["Percent Change"].shift(i)

    # candlestick_data[f"Day_{i} SP500 Open"] = data_P_index["Open"].shift(i)
    # candlestick_data[f"Day_{i} SP500 High"] = data_P_index["High"].shift(i)
    # candlestick_data[f"Day_{i} SP500 Low"] = data_P_index["Low"].shift(i)
    # candlestick_data[f"Day_{i} SP500 Adj Close"] = data_P_index["Adj Close"].shift(i)
    # candlestick_data[f"Day_{i} SP500 Volume"] = data_P_index["Volume"].shift(i)
    # candlestick_data[f"Day_{i} SP500 Percent Change"] = data_P_index["Percent Change"].shift(i)

    # candlestick_data[f"Day_{i} QQQ Open"] = data_S_index["Open"].shift(i)
    # candlestick_data[f"Day_{i} QQQ High"] = data_S_index["High"].shift(i)
    # candlestick_data[f"Day_{i} QQQ Low"] = data_S_index["Low"].shift(i)
    # candlestick_data[f"Day_{i} QQQ Adj Close"] = data_S_index["Adj Close"].shift(i)
    # candlestick_data[f"Day_{i} QQQ Volume"] = data_S_index["Volume"].shift(i)
    # candlestick_data[f"Day_{i} QQQ Percent Change"] = data_S_index["Percent Change"].shift(i)

    # candlestick_data[f"Day_{i} corr Open"] = data_corr["Open"].shift(i)
    # candlestick_data[f"Day_{i} corr High"] = data_corr["High"].shift(i)
    # candlestick_data[f"Day_{i} corr Low"] = data_corr["Low"].shift(i)
    # candlestick_data[f"Day_{i} corr Adj Close"] = data_corr["Adj Close"].shift(i)
    # candlestick_data[f"Day_{i} corr Volume"] = data_corr["Volume"].shift(i)
    # candlestick_data[f"Day_{i} corr Percent Change"] = data_corr["Percent Change"].shift(i)

    # candlestick_data[f"Day_{i} industry Open"] = data_industry["Open"].shift(i)
    # candlestick_data[f"Day_{i} industry High"] = data_industry["High"].shift(i)
    # candlestick_data[f"Day_{i} industry Low"] = data_industry["Low"].shift(i)
    # candlestick_data[f"Day_{i} industry Adj Close"] = data_industry["Adj Close"].shift(i)
    # candlestick_data[f"Day_{i} industry Volume"] = data_industry["Volume"].shift(i)
    # candlestick_data[f"Day_{i} industry Percent Change"] = data_industry["Percent Change"].shift(i)

def categorize_movement(change):
    if change <= -10.0:
        return -3
    elif change <= -5.0:
        return -2
    elif change <= -2.5:
        return -1
    elif change >= 10.0:
        return 3
    elif change >= 5.0:
        return 2
    elif change >= 2.5:
        return 1
    else:
        return 0

candlestick_data["Movement"] = data_ticker["5 Day Percent Change"].apply(categorize_movement)
candlestick_data = candlestick_data.dropna().reset_index(drop=True)

frequency = candlestick_data["Movement"].value_counts()
print(frequency)

[*********************100%***********************]  1 of 1 completed

Movement
-3    135
 0    134
-2    121
 2     99
 3     97
-1     56
 1     55
Name: count, dtype: int64



/var/folders/12/hdw_ls8n7rs0lj0nn0x94wgc0000gn/T/ipykernel_56567/76269509.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ticker["5 Day Percent Change"] = ((data_ticker["Close"].shift(-5) - data_ticker["Close"])/data_ticker["Close"]) * 100


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

test_size = 252
train_df = candlestick_data.iloc[:-test_size]
test_df = candlestick_data.iloc[-test_size:]

x_train = train_df.drop(columns=["Movement"])
y_train = train_df["Movement"]
x_test = test_df.drop(columns=["Movement"])
y_test = test_df["Movement"]

feature_number = x_train.shape[1]
print(feature_number)

model = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 250, 500, 1000],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5],
    'bootstrap': [True],
    'max_features': ['sqrt', 'log2', 0.5, 0.7, 0.9]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2)

grid_search.fit(x_train, y_train)
y_pred = grid_search.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


60
Fitting 5 folds for each of 160 candidates, totalling 800 fits
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=250; total time=   0.6s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=250; total time=   0.7s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_split=2, n_estimators=250; total tim

In [34]:
from sklearn.metrics import confusion_matrix

y_pred = grid_search.predict(x_test)
cm = confusion_matrix(y_test, y_pred)

TP = {}
FP = {}
FN = {}
TN = {}

# Calculate the metrics for each class
for i in range(len(cm)):
    TP[i] = cm[i, i]
    FP[i] = sum(cm[:, i]) - cm[i, i]
    FN[i] = sum(cm[i, :]) - cm[i, i]
    TN[i] = cm.sum() - (FP[i] + FN[i] + TP[i])

# Output the results
for i in range(len(cm)):
    print(f"Class {i}:")
    print(f"TP: {TP[i]}, FP: {FP[i]}, TN: {TN[i]}, FN: {FN[i]}\n")

Class 0:
TP: 0, FP: 1, TN: 215, FN: 36

Class 1:
TP: 7, FP: 30, TN: 171, FN: 44

Class 2:
TP: 0, FP: 3, TN: 231, FN: 18

Class 3:
TP: 7, FP: 43, TN: 153, FN: 49

Class 4:
TP: 0, FP: 0, TN: 229, FN: 23

Class 5:
TP: 13, FP: 78, TN: 138, FN: 23

Class 6:
TP: 5, FP: 65, TN: 155, FN: 27

